In [12]:
import os
import tensorflow as tf
from keras import models
from keras import layers
from keras import Sequential
from datetime import datetime
from PIL import Image
import numpy as np
import keras.backend as K
import matplotlib.pyplot as plt
import mlflow
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.utils import plot_model
from matplotlib import pyplot as plt
from keras.models import Model
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, precision_score, recall_score, accuracy_score

In [13]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set the path to your dataset directory

dataset_dir = "dataset_2_final"



In [14]:

# Create the ImageDataGenerator with data augmentation and cropping
train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # Splitting the dataset into training and validation sets
)


In [15]:

# Custom data generator to generate four cropped images per input image

def custom_data_generator(data_generator, steps_per_epoch):
    i = 0  # Initialize the batch counter
    while True:  # Create an infinite loop for the generator
        batch_images = []
        batch_labels = []
        for images, labels in data_generator:
            for image, label in zip(images, labels):
                # Crop the image into four quadrants
                height, width = image.shape[0], image.shape[1]
                half_height, half_width = height // 2, width // 2
                quadrants = [
                    image[:half_height, :half_width, :],
                    image[:half_height, half_width:, :],
                    image[half_height:, :half_width, :],
                    image[half_height:, half_width:, :],
                ]
                batch_images.extend(quadrants)
                batch_labels.extend([label] * 4)  # Fix the label repetition for each quadrant

                # Check if the batch is large enough
                if len(batch_images) >= batch_size:  # Assuming you have a 'batch_size' variable defined
                    break

            # Check if we have generated enough batches (steps_per_epoch)
            if i >= steps_per_epoch:
                return  # End the generator

        batch_images = np.stack(batch_images)
        batch_labels = np.stack(batch_labels)
        yield batch_images, batch_labels

        i += 1  # Increment the batch counter


In [16]:
image_size = (300, 300)
batch_size = 4

In [17]:

# Load the training dataset using image_dataset_from_directory
train_ds = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='sparse',  # For integer labels
    shuffle=True,
    subset='training',    # Selecting the training set
    seed=1337
)

# Load the validation dataset using image_dataset_from_directory
val_ds = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='sparse',  # For integer labels
    shuffle=False,        # No need to shuffle the validation set
    subset='validation',  # Selecting the validation set
    seed=1337
)

# Check the number of classes in your dataset
num_classes = len(train_ds.class_indices)


Found 423 images belonging to 7 classes.
Found 102 images belonging to 7 classes.


In [18]:
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense

In [19]:

# Initialize the EfficientNetB3 model
base_model = EfficientNetB3(weights='imagenet', include_top=False, input_shape=image_size + (3,))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)


In [20]:
# Train the model using the custom data generator
epochs = 20



In [24]:
# Calculate the total number of training and validation images
total_train_images = len(train_ds.filenames)
total_val_images = len(val_ds.filenames)

# Calculate steps_per_epoch and validation_steps
steps_per_epoch = total_train_images // batch_size
validation_steps = total_val_images // batch_size

# If there are any remaining images that do not fit in a full batch, add one more step
if total_train_images % batch_size != 0:
    steps_per_epoch += 1
if total_val_images % batch_size != 0:
    validation_steps += 1


In [26]:
print(total_train_images)

423


In [28]:

# Train the model using the custom data generator
model.fit(
    custom_data_generator(train_ds, steps_per_epoch),  # Pass the custom data generator here
    validation_data=val_ds,
    epochs=epochs,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps
)